In [27]:
import numpy as np
import pandas as pd
import yfinance as yf
tickers = ["SPY", "TLT"]
data = yf.download(tickers, start="2021-01-01", end="2025-12-31")["Close"].dropna()
log_prices = np.log(data.values)     
R = np.diff(log_prices, axis=0)        

/var/folders/_b/28jfj19121qf32rdzd8twc000000gn/T/ipykernel_25819/2775713885.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start="2021-01-01", end="2025-12-31")["Close"].dropna()
[*********************100%***********************]  2 of 2 completed


In [28]:
R

array([[ 0.00686409, -0.00745402],
       [ 0.00596059, -0.02074191],
       [ 0.014748  , -0.0088541 ],
       ...,
       [ 0.00345679,  0.00281104],
       [ 0.00073084, -0.00540415],
       [ 0.00189768, -0.00463937]])

In [29]:
T, k = R.shape

In [30]:
Y = R[1:,]

In [31]:
Y

array([[ 0.00596059, -0.02074191],
       [ 0.014748  , -0.0088541 ],
       [ 0.00568157, -0.0032324 ],
       ...,
       [ 0.00345679,  0.00281104],
       [ 0.00073084, -0.00540415],
       [ 0.00189768, -0.00463937]])

In [32]:
lag1 = R[:-1,]

In [33]:
lag1

array([[ 0.00686409, -0.00745402],
       [ 0.00596059, -0.02074191],
       [ 0.014748  , -0.0088541 ],
       ...,
       [ 0.00185051,  0.00045051],
       [ 0.00345679,  0.00281104],
       [ 0.00073084, -0.00540415]])

In [34]:
intercep = np.ones((T-1,1))

In [35]:
intercep

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]])

In [36]:
X = np.hstack([intercep,lag1])

In [37]:
X

array([[ 1.00000000e+00,  6.86408902e-03, -7.45401649e-03],
       [ 1.00000000e+00,  5.96059087e-03, -2.07419056e-02],
       [ 1.00000000e+00,  1.47479971e-02, -8.85409917e-03],
       ...,
       [ 1.00000000e+00,  1.85050680e-03,  4.50511516e-04],
       [ 1.00000000e+00,  3.45679370e-03,  2.81104370e-03],
       [ 1.00000000e+00,  7.30844581e-04, -5.40415331e-03]])

In [38]:
Y.shape

(1236, 2)

In [39]:
X.shape

(1236, 3)

In [40]:
XtX = X.T @ X
Xty = X.T @ Y
B_hat = np.linalg.solve(XtX,Xty)

In [41]:
B_hat

array([[ 0.00055932, -0.00038197],
       [-0.02816221,  0.04655567],
       [-0.0169606 , -0.04714999]])

In [42]:
c_hat = B_hat[0,:]

In [43]:
c_hat

array([ 0.00055932, -0.00038197])

In [46]:
A1_hat = B_hat[1:,:].T

In [47]:
a11, a12 = A1_hat[0, 0], A1_hat[0, 1]
a21, a22 = A1_hat[1, 0], A1_hat[1, 1]

print("a_12 (TLT_{t-1} -> SPY_t):", a12)
print("a_21 (SPY_{t-1} -> TLT_t):", a21)

a_12 (TLT_{t-1} -> SPY_t): -0.016960604415754598
a_21 (SPY_{t-1} -> TLT_t): 0.04655567267290832


In [49]:
import numpy as np
import pandas as pd
import yfinance as yf

# 1) Get data and returns
tickers = ["SPY", "TLT"]
data = yf.download(tickers, start="2015-01-01", end="2025-01-01")["Close"].dropna()
log_prices = np.log(data.values)          # (T, 2)
R = np.diff(log_prices, axis=0)           # (T-1, 2)

# 2) Build VAR(1) matrices
def build_var1_matrices(R):
    T, k = R.shape
    Y = R[1:, :]               # (T-1, k)
    intercept = np.ones((T-1, 1))
    lag1 = R[:-1, :]           # (T-1, k)
    X = np.hstack([intercept, lag1])  # (T-1, 1 + k)
    return Y, X

Y, X = build_var1_matrices(R)

# 3) OLS estimate of B
XtX = X.T @ X                      # (m, m)
XtY = X.T @ Y                      # (m, k)
XtX_inv = np.linalg.inv(XtX)
B_hat = XtX_inv @ XtY              # (m, k)

m, k = B_hat.shape   # m = 3 (intercept, SPY_{t-1}, TLT_{t-1}), k = 2 (SPY, TLT)

# 4) Residuals and residual covariance
Y_hat = X @ B_hat                  # (T-1, k)
U_hat = Y - Y_hat                  # residuals

T_eff = Y.shape[0]
df = T_eff - m                     # degrees of freedom
Sigma_u_hat = (U_hat.T @ U_hat) / df   # (k, k)

# 5) Standard errors for each column (equation-wise)
se_B = np.zeros_like(B_hat)        # same shape as B_hat

diag_XtX_inv = np.diag(XtX_inv)    # (m,)

for j in range(k):  # j = 0: SPY eq, j = 1: TLT eq
    sigma2_j = Sigma_u_hat[j, j]
    se_B[:, j] = np.sqrt(sigma2_j * diag_XtX_inv)

# 6) 95% confidence intervals
z = 1.96
lower = B_hat - z * se_B
upper = B_hat + z * se_B

# For readability: label rows and columns
rows = ["intercept", "lag_SPY", "lag_TLT"]
cols = ["SPY_eq", "TLT_eq"]

coef_df = pd.DataFrame(B_hat, index=rows, columns=cols)
se_df   = pd.DataFrame(se_B,   index=rows, columns=cols)
low_df  = pd.DataFrame(lower,  index=rows, columns=cols)
up_df   = pd.DataFrame(upper,  index=rows, columns=cols)

print("Coefficients:\n", coef_df)
print("\nStd. errors:\n", se_df)
print("\n95% CI lower:\n", low_df)
print("\n95% CI upper:\n", up_df)

/var/folders/_b/28jfj19121qf32rdzd8twc000000gn/T/ipykernel_25819/3616912868.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start="2015-01-01", end="2025-01-01")["Close"].dropna()
[*********************100%***********************]  2 of 2 completed

Coefficients:
              SPY_eq    TLT_eq
intercept  0.000547 -0.000092
lag_SPY   -0.104157  0.073111
lag_TLT    0.065423 -0.008547

Std. errors:
              SPY_eq    TLT_eq
intercept  0.000220  0.000192
lag_SPY    0.020227  0.017600
lag_TLT    0.023355  0.020322

95% CI lower:
              SPY_eq    TLT_eq
intercept  0.000115 -0.000468
lag_SPY   -0.143803  0.038615
lag_TLT    0.019646 -0.048377

95% CI upper:
              SPY_eq    TLT_eq
intercept  0.000980  0.000284
lag_SPY   -0.064512  0.107607
lag_TLT    0.111199  0.031283


In [52]:

A1_hat = B_hat[1:,:].T
eigvals = np.linalg.eigvals(A1_hat)
print("Eigenvalues of A_hat:", eigvals)
print("Moduli:", np.abs(eigvals))

Eigenvalues of A_hat: [-0.14042635  0.02772187]
Moduli: [0.14042635 0.02772187]


In [53]:
# SPY equation is column 0
coef_lag_TLT_on_SPY = B_hat[2, 0]
se_lag_TLT_on_SPY   = se_B[2, 0]
t_stat = coef_lag_TLT_on_SPY / se_lag_TLT_on_SPY

In [54]:
t_stat

2.8011871139233584

In [56]:
# A_hat from before
H = 10  # horizon
# Cholesky-orthogonalized shocks (optional)
# For simplicity, start with identity: shock only SPY = [1, 0]
shock = np.array([1.0, 0.0])  # instant shock at SPY

irf = np.zeros((H+1, 2))  # IRF[h, :] = response at horizon h
irf[0, :] = shock

A_power = np.eye(2)
for h in range(1, H+1):
    A_power = A1_hat @ A_power
    irf[h, :] = A_power @ shock

print("IRF (rows=horizon, cols=[SPY, TLT]):\n", irf)

IRF (rows=horizon, cols=[SPY, TLT]):
 [[ 1.00000000e+00  0.00000000e+00]
 [-1.04157254e-01  7.31112537e-02]
 [ 1.56318697e-02 -8.23996566e-03]
 [-2.16725347e-03  1.21329442e-03]
 [ 3.05112175e-04 -1.68820917e-04]
 [-4.28243676e-05  2.37500837e-05]
 [ 6.01426328e-06 -3.33394047e-06]
 [-8.44544557e-07  4.68206263e-07]
 [ 1.18596763e-07 -6.57475749e-08]
 [-1.66540974e-08  9.23271723e-09]
 [ 2.33867441e-09 -1.29651604e-09]]
